In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

In [2]:
import keras_tuner as kt
from keras_tuner import RandomSearch
# from GEH_LOSS import GEH_loss
import geh as ga

In [3]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error

In [4]:
import relative_accuracy as ra

In [5]:
# Load all Inputs and Output Data

# 5 mins (1 step ahead)
Deep_train_5   = np.load("Deep_train_5_multi.npz")['x'] 
Output_train_5 = np.load("Deep_train_5_multi.npz")['y']

Deep_test_5   = np.load("Deep_test_5_multi.npz")['x'] 
Output_test_5 = np.load("Deep_test_5_multi.npz")['y'] 

# 15 mins (3 steps ahead)
Deep_train_15   = np.load("Deep_train_15_multi.npz")['x'] 
Output_train_15 = np.load("Deep_train_15_multi.npz")['y']

Deep_test_15   = np.load("Deep_test_15_multi.npz")['x'] 
Output_test_15 = np.load("Deep_test_15_multi.npz")['y']

# 30 mins (6 steps ahead)
Deep_train_30   = np.load("Deep_train_30_multi.npz")['x'] 
Output_train_30 = np.load("Deep_train_30_multi.npz")['y']

Deep_test_30   = np.load("Deep_test_30_multi.npz")['x'] 
Output_test_30 = np.load("Deep_test_30_multi.npz")['y']

# 60 mins (12 steps ahead)
Deep_train_60   = np.load("Deep_train_60_multi.npz")['x'] 
Output_train_60 = np.load("Deep_train_60_multi.npz")['y']

Deep_test_60   = np.load("Deep_test_60_multi.npz")['x'] 
Output_test_60 = np.load("Deep_test_60_multi.npz")['y']

In [6]:
# Wide Data
Xtrain = joblib.load("002weeks_train_Multi.save") 
Xtest = joblib.load("002weeks_test_Multi.save") 

In [7]:
# 5 mins
# Delete first 15 samples
Wide_train_5 = np.delete(Xtrain, np.s_[0:15], 0)
Wide_test_5 = np.delete(Xtest, np.s_[0:15], 0)

In [8]:
# 15 mins
# Delete first 17 samples
Wide_train_15 = np.delete(Xtrain, np.s_[0:17], 0)
Wide_test_15 = np.delete(Xtest, np.s_[0:17], 0)

In [9]:
# 30 mins
# Delete first 20 samples
Wide_train_30 = np.delete(Xtrain, np.s_[0:20], 0)
Wide_test_30 = np.delete(Xtest, np.s_[0:20], 0)

In [10]:
# 60 mins
# Delete first 27 samples
Wide_train_60 = np.delete(Xtrain, np.s_[0:26], 0)
Wide_test_60 = np.delete(Xtest, np.s_[0:26], 0)

In [11]:
scaler_filename = "scaler.save"
scaler = joblib.load(scaler_filename) 

C:\Users\mesug\anaconda3\envs\my_env\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
Test      = pd.read_csv('01test_scaled.csv', index_col=None, parse_dates=[0])

In [13]:
from pyts.image import GramianAngularField

# Reshape Samples

The first step is to split the input sequences into subsequences that can be processed by the CNN model. Here, each spatio-temporal sample can be split into three sub-samples, each with five time steps. The CNN can interpret each subsequence of five time steps and provide a time series of interpretations of the subsequences to the LSTM model to process as input.

In [14]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 7                # No of loop detectors
n_seq = 3                     # Subsequences
n_steps = 5                   # time-step per subsequence
# val_percent = 0.07567       # 2 weeks
val_percent = 0.2        
batch_size = 32
n_GAF = 15

n_seq_ = 3                     # Subsequences
n_steps_ = 3                   # time-step per subsequence

In [15]:
# Summation
gasf = GramianAngularField(image_size=1., method='summation', sample_range=None)
# Difference
gadf = GramianAngularField(image_size=1., method='difference', sample_range=None)

#### 5 mins

In [16]:
# Train
Xt5 = np.empty((len(Deep_train_5), n_features, n_GAF, n_GAF))
yt5 = scaler.inverse_transform(Output_train_5)

In [17]:
# Test
Xv5 = np.empty((len(Deep_test_5), n_features, n_GAF, n_GAF))
yv5 = scaler.inverse_transform(Output_test_5)

#### 15 mins

In [18]:
# Train
Xt15 = np.empty((len(Deep_train_15), n_features, n_GAF, n_GAF))
yt15 = scaler.inverse_transform(Output_train_15)

In [19]:
# Test
Xv15 = np.empty((len(Deep_test_15), n_features, n_GAF, n_GAF))
yv15 = scaler.inverse_transform(Output_test_15)

#### 30 mins

In [20]:
# Train
Xt30 = np.empty((len(Deep_train_30), n_features, n_GAF, n_GAF))
yt30 = scaler.inverse_transform(Output_train_30)

In [21]:
# Test
Xv30 = np.empty((len(Deep_test_30), n_features, n_GAF, n_GAF))
yv30 = scaler.inverse_transform(Output_test_30)

#### 60 mins

In [22]:
# Train
Xt60 = np.empty((len(Deep_train_60), n_features, n_GAF, n_GAF))
yt60 = scaler.inverse_transform(Output_train_60)

In [23]:
# Test
Xv60 = np.empty((len(Deep_test_60), n_features, n_GAF, n_GAF))
yv60 = scaler.inverse_transform(Output_test_60)

In [24]:
## Check Size

In [25]:
Xt5.shape

(31089, 7, 15, 15)

## Tranformations

### 5 mins

In [26]:
for i in range(len(Xt5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_5[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt5[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [27]:
for i in range(len(Xv5)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_5[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv5[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [28]:
Xt5.shape

(31089, 7, 15, 15)

In [29]:
Xv5.shape

(4017, 7, 15, 15)

### 15 mins

In [30]:
for i in range(len(Xt15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_15[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt15[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [31]:
for i in range(len(Xv15)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_15[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv15[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [32]:
Xt15.shape

(31087, 7, 15, 15)

In [33]:
Xv15.shape

(4015, 7, 15, 15)

### 30 mins

In [34]:
for i in range(len(Xt30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_30[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt30[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [35]:
for i in range(len(Xv30)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_30[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv30[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [36]:
Xt30.shape

(31084, 7, 15, 15)

In [37]:
Xv30.shape

(4012, 7, 15, 15)

### 60 mins

In [38]:
for i in range(len(Xt60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_train_60[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xt30[i,:,:,:] = X_gasf
    Xt5[i,:,:,:] = X_gadf

In [39]:
for i in range(len(Xv60)):
    # Transpose data first because:
    # expected shape is (n_samples, n_timestamps)
    X = np.transpose(Deep_test_60[i])
    
    # Perform the GAF transformations
    # X_gasf = gasf.transform(X)
    X_gadf = gadf.transform(X)
    
    # Store value in the array to get the shape:
    # (samples, feature, n_GAF, n_GAF)
    # Xv60[i,:,:,:] = X_gasf
    Xv5[i,:,:,:] = X_gadf

In [40]:
Xt60.shape

(31078, 7, 15, 15)

In [41]:
Xv60.shape

(4006, 7, 15, 15)

### Reshaping the Data for Conv2D layer

For Conv2D, there is a need to add one more dimension to show we're dealing with 1 channel (since technically the images are in black and white, only showing values from 0-max flow on a single channel).

Conv1D - strides in 1 dimension
Conv2D - strides in 2 dimensions

In [42]:
# Train data
Xt5  =  np.transpose(Xt5, (0, 2, 3, 1))
Xt15 =  np.transpose(Xt15, (0, 2, 3, 1))
Xt30 =  np.transpose(Xt30, (0, 2, 3, 1))
Xt60 =  np.transpose(Xt60, (0, 2, 3, 1))

# Validation data
Xv5  =  np.transpose(Xv5, (0, 2, 3, 1))
Xv15 =  np.transpose(Xv15, (0, 2, 3, 1))
Xv30 =  np.transpose(Xv30, (0, 2, 3, 1))
Xv60 =  np.transpose(Xv60, (0, 2, 3, 1))

In [43]:
Xt5.shape

(31089, 15, 15, 7)

In [44]:
Xv5.shape

(4017, 15, 15, 7)

In [45]:
# Train data
Xt5  =  Xt5.reshape(Xt5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt15 =  Xt15.reshape(Xt15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt30 =  Xt30.reshape(Xt30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xt60 =  Xt60.reshape(Xt60.shape[0], n_seq, n_steps, n_GAF, n_features)

# Validation data
Xv5  =  Xv5.reshape(Xv5.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv15 =  Xv15.reshape(Xv15.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv30 =  Xv30.reshape(Xv30.shape[0], n_seq, n_steps, n_GAF, n_features)
Xv60 =  Xv60.reshape(Xv60.shape[0], n_seq, n_steps, n_GAF, n_features)

In [46]:
Wt5  = Wide_train_5
Wt15 = Wide_train_15
Wt30 = Wide_train_30
Wt60 = Wide_train_60

Wv5  = Wide_test_5
Wv15 = Wide_test_15
Wv30 = Wide_test_30
Wv60 = Wide_test_60

In [47]:
# Train data
Wt5  =  Wt5.reshape(Wt5.shape[0], n_seq_, n_steps_, n_features)
Wt15 =  Wt15.reshape(Wt15.shape[0], n_seq_, n_steps_, n_features)
Wt30 =  Wt30.reshape(Wt30.shape[0], n_seq_, n_steps_, n_features)
Wt60 =  Wt60.reshape(Wt60.shape[0], n_seq_, n_steps_, n_features)

# Validation data
Wv5  =  Wv5.reshape(Wv5.shape[0], n_seq_, n_steps_, n_features)
Wv15 =  Wv15.reshape(Wv15.shape[0], n_seq_, n_steps_, n_features)
Wv30 =  Wv30.reshape(Wv30.shape[0], n_seq_, n_steps_, n_features)
Wv60 =  Wv60.reshape(Wv60.shape[0], n_seq_, n_steps_, n_features)

In [48]:
Xt5.shape

(31089, 3, 5, 15, 7)

In [49]:
Wt5.shape

(31089, 3, 3, 7)

# CNN-LSTM model

In [50]:
from tensorflow import keras
from keras_self_attention import SeqSelfAttention

# Early Stopping
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=5, restore_best_weights = True)

In [51]:
Wide_train_5.shape

(31089, 9, 7)

# 5 mins ahead

In [52]:
def build_model_5(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt5.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt5.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_5w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_5 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_5w)
    
    
    # Deep Model
    # CNN 
    CNN_5 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_5 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_5)
    # LSTM          
    LSTM_5 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_5)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_5, LSTM_5], axis=1)

    # Attention
    Att_5 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_5 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_5.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                                   
    return model_5

In [53]:
tuner5 = RandomSearch(
    build_model_5,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10b-5",
)

INFO:tensorflow:Reloading Tuner from C:\Runs\10b-5\tuner0.json


In [54]:
tuner5.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [55]:
tuner5.search((Wt5,Xt5), yt5, epochs=200,
           validation_split = val_percent,
           callbacks = [early_stop],
           verbose=2)

Trial 10 Complete [02h 37m 35s]
val_loss: 925.1968994140625

Best val_loss So Far: 894.143798828125
Total elapsed time: 02h 37m 35s
INFO:tensorflow:Oracle triggered exit


In [56]:
tuner5.results_summary(1)

Results summary
Results in C:\Runs\10b-5
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 01 summary
Hyperparameters:
units: 82
filters: 512
kernel_size: 2
learning_rate: 0.001
Score: 894.143798828125


### Best Model

In [58]:
best_hp5 = tuner5.get_best_hyperparameters()[0]
model5 = tuner5.hypermodel.build(best_hp5)

In [59]:
scaler_filename = "Best_HP/10b-5"
joblib.dump(best_hp5, scaler_filename) 

['Best_HP/10b-5']

In [60]:
scaler_filename = "Best_HP/10b-5"
best_hp5 = joblib.load(scaler_filename) 

model5 = tuner5.hypermodel.build(best_hp5)

In [61]:
model5.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 512 14848       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [62]:
model5.fit((Wt5,Xt5), yt5, validation_split = val_percent,
           epochs = 100, 
           verbose = 1)

Epoch 1/100
778/778 [==============================] - 82s 95ms/step - loss: 20490.0625 - mean_absolute_error: 105.8178 - root_mean_squared_error: 143.1435 - mean_absolute_percentage_error: 53304808.0000 - val_loss: 4313.2690 - val_mean_absolute_error: 45.8287 - val_root_mean_squared_error: 65.6755 - val_mean_absolute_percentage_error: 4515947.0000
Epoch 2/100
778/778 [==============================] - 71s 91ms/step - loss: 2261.7578 - mean_absolute_error: 32.1405 - root_mean_squared_error: 47.5579 - mean_absolute_percentage_error: 50708328.0000 - val_loss: 1406.6656 - val_mean_absolute_error: 26.5593 - val_root_mean_squared_error: 37.5056 - val_mean_absolute_percentage_error: 3777687.5000
Epoch 3/100
778/778 [==============================] - 70s 90ms/step - loss: 1063.8967 - mean_absolute_error: 23.2931 - root_mean_squared_error: 32.6174 - mean_absolute_percentage_error: 53618436.0000 - val_loss: 1052.3053 - val_mean_absolute_error: 23.1067 - val_root_mean_squared_error: 32.4393 - va

778/778 [==============================] - 68s 88ms/step - loss: 145.9721 - mean_absolute_error: 8.6876 - root_mean_squared_error: 12.0819 - mean_absolute_percentage_error: 13720936.0000 - val_loss: 978.2320 - val_mean_absolute_error: 22.8488 - val_root_mean_squared_error: 31.2767 - val_mean_absolute_percentage_error: 2455456.7500
Epoch 43/100
778/778 [==============================] - 65s 83ms/step - loss: 139.2252 - mean_absolute_error: 8.4643 - root_mean_squared_error: 11.7994 - mean_absolute_percentage_error: 13195538.0000 - val_loss: 977.5651 - val_mean_absolute_error: 22.9088 - val_root_mean_squared_error: 31.2660 - val_mean_absolute_percentage_error: 2403331.7500mean_squared_err - ETA: 30s - loss: 139.1668 - mean_ab
Epoch 44/100
778/778 [==============================] - 65s 84ms/step - loss: 133.3349 - mean_absolute_error: 8.2682 - root_mean_squared_error: 11.5471 - mean_absolute_percentage_error: 12480682.0000 - val_loss: 973.0284 - val_mean_absolute_error: 22.8111 - val_root_

Epoch 65/100
778/778 [==============================] - 67s 86ms/step - loss: 60.0754 - mean_absolute_error: 5.5770 - root_mean_squared_error: 7.7508 - mean_absolute_percentage_error: 4886740.5000 - val_loss: 1002.3110 - val_mean_absolute_error: 23.2158 - val_root_mean_squared_error: 31.6593 - val_mean_absolute_percentage_error: 2608053.0000
Epoch 66/100
778/778 [==============================] - 66s 85ms/step - loss: 56.7701 - mean_absolute_error: 5.4251 - root_mean_squared_error: 7.5346 - mean_absolute_percentage_error: 5241242.5000 - val_loss: 1007.1962 - val_mean_absolute_error: 23.2555 - val_root_mean_squared_error: 31.7364 - val_mean_absolute_percentage_error: 2746616.2500
Epoch 67/100
778/778 [==============================] - 68s 87ms/step - loss: 54.9050 - mean_absolute_error: 5.3367 - root_mean_squared_error: 7.4098 - mean_absolute_percentage_error: 4984793.5000 - val_loss: 1004.6406 - val_mean_absolute_error: 23.2322 - val_root_mean_squared_error: 31.6961 - val_mean_absolute

In [63]:
metrics5 = pd.DataFrame(model5.history.history)

In [64]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics5[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics5[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(400,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [65]:
model5.save("my_dir/10b-5")

INFO:tensorflow:Assets written to: my_dir/10b-5\assets


INFO:tensorflow:Assets written to: my_dir/10b-5\assets


In [66]:
model5 = keras.models.load_model("my_dir/10b-5") 

In [67]:
model5.predict((Wv5,Xv5))

array([[ 60.910927,  69.84143 ,  65.098206, ...,  63.897644,  30.598059,
         31.731276],
       [ 58.20304 ,  64.22784 ,  66.98384 , ...,  59.839355,  43.055237,
         52.91254 ],
       [ 63.51069 ,  70.82185 ,  63.362167, ...,  67.73981 ,  37.974625,
         43.659687],
       ...,
       [153.99297 , 156.36574 , 163.75438 , ..., 159.17992 ,  99.89022 ,
         91.90446 ],
       [135.96739 , 143.32318 , 133.14159 , ..., 121.71164 ,  81.23408 ,
         76.319176],
       [133.32362 , 122.78693 , 123.97352 , ..., 126.15914 ,  77.61408 ,
         78.52111 ]], dtype=float32)

# 15 mins ahead

In [70]:
def build_model_15(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt15.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt15.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_15w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_15 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_15w)
    
    
    
    # Deep Model
    # CNN 
    CNN_15 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_15 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_15)
    # LSTM          
    LSTM_15 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_15)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_15, LSTM_15], axis=1)

    # Attention
    Att_15 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_15 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_15.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_15

In [71]:
tuner15 = RandomSearch(
    build_model_15,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10b-15",
)

In [72]:
tuner15.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [73]:
tuner15.search((Wt15,Xt15), yt15, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop], 
            verbose=2)

Trial 10 Complete [00h 27m 39s]
val_loss: 1159.9854736328125

Best val_loss So Far: 1134.2809448242188
Total elapsed time: 17h 06m 36s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [74]:
tuner15.results_summary(1)

Results summary
Results in C:\Runs\10b-15
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 04 summary
Hyperparameters:
units: 77
filters: 416
kernel_size: 2
learning_rate: 0.001
Score: 1134.2809448242188


### Best model

In [75]:
best_hp15 = tuner15.get_best_hyperparameters()[0]
model15 = tuner15.hypermodel.build(best_hp15)

In [76]:
scaler_filename = "Best_HP/10b-15"
joblib.dump(best_hp15, scaler_filename) 

['Best_HP/10b-15']

In [77]:
scaler_filename = "Best_HP/10b-15"
best_hp15 = joblib.load(scaler_filename) 

model15 = tuner15.hypermodel.build(best_hp15)

In [78]:
model15.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 416 12064       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [79]:
model15.fit((Wt15,Xt15), yt15, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
778/778 [==============================] - 95s 103ms/step - loss: 21945.0703 - mean_absolute_error: 111.1085 - root_mean_squared_error: 148.1388 - mean_absolute_percentage_error: 55140580.0000 - val_loss: 4663.7451 - val_mean_absolute_error: 48.6495 - val_root_mean_squared_error: 68.2916 - val_mean_absolute_percentage_error: 5323634.5000
Epoch 2/100
778/778 [==============================] - 77s 99ms/step - loss: 2452.1526 - mean_absolute_error: 34.7113 - root_mean_squared_error: 49.5192 - mean_absolute_percentage_error: 67564112.0000 - val_loss: 1691.4919 - val_mean_absolute_error: 29.2214 - val_root_mean_squared_error: 41.1277 - val_mean_absolute_percentage_error: 5040957.0000
Epoch 3/100
778/778 [==============================] - 78s 100ms/step - loss: 1157.6497 - mean_absolute_error: 24.7916 - root_mean_squared_error: 34.0243 - mean_absolute_percentage_error: 65228708.0000 - val_loss: 1320.2103 - val_mean_absolute_error: 25.4947 - val_root_mean_squared_error: 36.3347 - 

778/778 [==============================] - 76s 98ms/step - loss: 746.6443 - mean_absolute_error: 19.8855 - root_mean_squared_error: 27.3248 - mean_absolute_percentage_error: 57944000.0000 - val_loss: 1129.4431 - val_mean_absolute_error: 22.7617 - val_root_mean_squared_error: 33.6072 - val_mean_absolute_percentage_error: 3948446.7500
Epoch 48/100
778/778 [==============================] - 79s 101ms/step - loss: 746.8597 - mean_absolute_error: 19.8865 - root_mean_squared_error: 27.3287 - mean_absolute_percentage_error: 58146964.0000 - val_loss: 1137.9865 - val_mean_absolute_error: 22.8689 - val_root_mean_squared_error: 33.7341 - val_mean_absolute_percentage_error: 4080136.0000
Epoch 49/100
778/778 [==============================] - 77s 99ms/step - loss: 747.5010 - mean_absolute_error: 19.8933 - root_mean_squared_error: 27.3405 - mean_absolute_percentage_error: 58035988.0000 - val_loss: 1118.8898 - val_mean_absolute_error: 22.6684 - val_root_mean_squared_error: 33.4498 - val_mean_absolute

778/778 [==============================] - 79s 101ms/step - loss: 719.5572 - mean_absolute_error: 19.5202 - root_mean_squared_error: 26.8246 - mean_absolute_percentage_error: 57607252.0000 - val_loss: 1126.4236 - val_mean_absolute_error: 22.7226 - val_root_mean_squared_error: 33.5622 - val_mean_absolute_percentage_error: 4102652.7500
Epoch 94/100
778/778 [==============================] - 79s 101ms/step - loss: 721.3835 - mean_absolute_error: 19.5532 - root_mean_squared_error: 26.8586 - mean_absolute_percentage_error: 57651548.0000 - val_loss: 1140.5187 - val_mean_absolute_error: 22.7942 - val_root_mean_squared_error: 33.7716 - val_mean_absolute_percentage_error: 4111337.0000
Epoch 95/100
778/778 [==============================] - 77s 99ms/step - loss: 718.9587 - mean_absolute_error: 19.5080 - root_mean_squared_error: 26.8134 - mean_absolute_percentage_error: 57725056.0000 - val_loss: 1128.1670 - val_mean_absolute_error: 22.6965 - val_root_mean_squared_error: 33.5882 - val_mean_absolut

In [80]:
metrics15 = pd.DataFrame(model15.history.history)

In [81]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics15[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics15[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# #plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [82]:
model15.save("my_dir/10b-15")

INFO:tensorflow:Assets written to: my_dir/10b-15\assets


INFO:tensorflow:Assets written to: my_dir/10b-15\assets


In [83]:
model15 = keras.models.load_model("my_dir/10b-15")

In [84]:
model15.predict((Wv15,Xv15))

array([[ 56.3294  ,  57.13089 ,  62.94393 , ...,  60.412666,  36.53721 ,
         36.402695],
       [ 54.98835 ,  55.844543,  61.73107 , ...,  60.1453  ,  36.57093 ,
         36.266113],
       [ 54.773827,  55.60002 ,  61.74218 , ...,  61.147087,  37.167778,
         36.83461 ],
       ...,
       [143.08606 , 147.20358 , 159.00183 , ..., 157.11948 , 103.34178 ,
         94.21352 ],
       [141.83574 , 146.04422 , 157.77507 , ..., 156.96744 , 101.5801  ,
         93.20054 ],
       [127.53736 , 131.50441 , 142.06848 , ..., 139.32704 ,  91.05906 ,
         83.75151 ]], dtype=float32)

# 30 mins ahead

In [87]:
def build_model_30(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt30.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt30.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_30w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_30 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_30w)
    
    
    # Deep Model
    # CNN 
    CNN_30 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_30 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_30)
    # LSTM          
    LSTM_30 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_30)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_30, LSTM_30], axis=1)

    # Attention
    Att_30 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_30 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_30.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_30

In [88]:
tuner30 = RandomSearch(
    build_model_30,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10b-30",
)

In [89]:
tuner30.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [90]:
tuner30.search((Wt30,Xt30), yt30, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [00h 49m 42s]
val_loss: 1152.48388671875

Best val_loss So Far: 1131.497802734375
Total elapsed time: 13h 24m 59s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [91]:
tuner30.results_summary(1)

Results summary
Results in C:\Runs\10b-30
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 05 summary
Hyperparameters:
units: 53
filters: 416
kernel_size: 2
learning_rate: 0.01
Score: 1131.497802734375


### Best model

In [92]:
best_hp30 = tuner30.get_best_hyperparameters()[0]
model30 = tuner30.hypermodel.build(best_hp30)

In [93]:
scaler_filename = "Best_HP/10b-30"
joblib.dump(best_hp30, scaler_filename) 

['Best_HP/10b-30']

In [94]:
scaler_filename = "Best_HP/10b-30"
best_hp30 = joblib.load(scaler_filename) 

model30 = tuner30.hypermodel.build(best_hp30)

In [95]:
model30.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 4, 14, 416 12064       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [96]:
model30.fit((Wt30,Xt30), yt30, validation_split = val_percent,
            epochs = 100, 
            verbose = 1)

Epoch 1/100
778/778 [==============================] - 104s 126ms/step - loss: 4840.3267 - mean_absolute_error: 40.0596 - root_mean_squared_error: 69.5724 - mean_absolute_percentage_error: 64198740.0000 - val_loss: 1235.2156 - val_mean_absolute_error: 24.4156 - val_root_mean_squared_error: 35.1456 - val_mean_absolute_percentage_error: 4608397.5000
Epoch 2/100
778/778 [==============================] - 98s 125ms/step - loss: 893.9984 - mean_absolute_error: 21.8738 - root_mean_squared_error: 29.8998 - mean_absolute_percentage_error: 60785692.0000 - val_loss: 1211.1469 - val_mean_absolute_error: 23.9802 - val_root_mean_squared_error: 34.8015 - val_mean_absolute_percentage_error: 4342699.5000
Epoch 3/100
778/778 [==============================] - 95s 122ms/step - loss: 863.6465 - mean_absolute_error: 21.5116 - root_mean_squared_error: 29.3878 - mean_absolute_percentage_error: 60315384.0000 - val_loss: 1233.4315 - val_mean_absolute_error: 24.1249 - val_root_mean_squared_error: 35.1203 - val

778/778 [==============================] - 103s 133ms/step - loss: 758.8591 - mean_absolute_error: 20.0979 - root_mean_squared_error: 27.5474 - mean_absolute_percentage_error: 58315356.0000 - val_loss: 1126.6135 - val_mean_absolute_error: 22.8058 - val_root_mean_squared_error: 33.5651 - val_mean_absolute_percentage_error: 4056800.5000
Epoch 47/100
778/778 [==============================] - 100s 128ms/step - loss: 755.0113 - mean_absolute_error: 20.0323 - root_mean_squared_error: 27.4775 - mean_absolute_percentage_error: 58379348.0000 - val_loss: 1160.2887 - val_mean_absolute_error: 23.1492 - val_root_mean_squared_error: 34.0630 - val_mean_absolute_percentage_error: 3900544.2500
Epoch 48/100
778/778 [==============================] - 106s 136ms/step - loss: 755.0605 - mean_absolute_error: 20.0426 - root_mean_squared_error: 27.4784 - mean_absolute_percentage_error: 58135596.0000 - val_loss: 1142.9834 - val_mean_absolute_error: 22.9634 - val_root_mean_squared_error: 33.8080 - val_mean_abs

778/778 [==============================] - 103s 132ms/step - loss: 744.7569 - mean_absolute_error: 19.8832 - root_mean_squared_error: 27.2902 - mean_absolute_percentage_error: 57843328.0000 - val_loss: 1149.7130 - val_mean_absolute_error: 23.2448 - val_root_mean_squared_error: 33.9074 - val_mean_absolute_percentage_error: 4165349.7500
Epoch 93/100
778/778 [==============================] - 98s 126ms/step - loss: 745.6946 - mean_absolute_error: 19.9021 - root_mean_squared_error: 27.3074 - mean_absolute_percentage_error: 57812892.0000 - val_loss: 1145.3585 - val_mean_absolute_error: 23.1688 - val_root_mean_squared_error: 33.8431 - val_mean_absolute_percentage_error: 4229565.5000
Epoch 94/100
778/778 [==============================] - 102s 131ms/step - loss: 743.0249 - mean_absolute_error: 19.8799 - root_mean_squared_error: 27.2585 - mean_absolute_percentage_error: 57921732.0000 - val_loss: 1147.9500 - val_mean_absolute_error: 22.9924 - val_root_mean_squared_error: 33.8814 - val_mean_abso

In [97]:
metrics30 = pd.DataFrame(model30.history.history)

In [98]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics30[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics30[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [99]:
model30.save("my_dir/10b-30")

INFO:tensorflow:Assets written to: my_dir/10b-30\assets


INFO:tensorflow:Assets written to: my_dir/10b-30\assets


In [100]:
model30 = keras.models.load_model("my_dir/10b-30")

In [101]:
model30.predict((Wv30,Xv30))

array([[ 55.39309 ,  55.7585  ,  62.583168, ...,  58.08758 ,  31.142359,
         31.483368],
       [ 51.659477,  51.858246,  58.35119 , ...,  54.793087,  29.16076 ,
         29.723816],
       [ 52.59452 ,  52.509705,  59.176918, ...,  56.69293 ,  31.886744,
         32.2641  ],
       ...,
       [130.54787 , 132.92613 , 144.84274 , ..., 135.17789 ,  90.315254,
         82.804474],
       [126.67985 , 129.11116 , 141.58563 , ..., 131.68103 ,  87.256294,
         79.60339 ],
       [117.58409 , 120.67651 , 131.24968 , ..., 122.61566 ,  76.83633 ,
         71.24238 ]], dtype=float32)

# 60 mins ahead

In [104]:
def build_model_60(hp):
    # Inputs
    channel_Wide = keras.layers.Input(shape=Wt60.shape[1:], name="WideInput")
    channel_Deep = keras.layers.Input(shape=Xt60.shape[1:], name="DeepInput")
    units = hp.Int("units", min_value=1, max_value=100, step=1)
    
    
    # Wide Model
    flatten_60w = keras.layers.TimeDistributed(keras.layers.Flatten())(channel_Wide)
    Bi_60 = keras.layers.Bidirectional(
        keras.layers.LSTM(units=units, return_sequences=True),
        merge_mode="ave"
    )(flatten_60w)
    
    
    # Deep Model
    # CNN 
    CNN_60 = keras.layers.TimeDistributed(
        keras.layers.Conv2D(filters=hp.Int("filters", min_value=32, max_value=512, step=32),
                            kernel_size=hp.Choice("kernel_size", [2, 3]), activation='relu'))(channel_Deep)
    flatten_60 = keras.layers.TimeDistributed(keras.layers.Flatten())(CNN_60)
    # LSTM          
    LSTM_60 = keras.layers.LSTM(units=units, return_sequences=True)(flatten_60)

    
    # Concatenation 
    concat = keras.layers.concatenate([Bi_60, LSTM_60], axis=1)

    # Attention
    Att_60 = SeqSelfAttention(attention_activation='sigmoid')(concat)
    merge = keras.layers.Flatten()(concat)

    
    # Output
    output = keras.layers.Dense(n_features, name= "output")(merge)
    
    # Model
    model_60 = keras.Model(inputs=[channel_Wide,channel_Deep], outputs=[output])

    # Compile
    model_60.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])), 
        loss="mse", metrics=['MeanAbsoluteError','RootMeanSquaredError','MeanAbsolutePercentageError'])
                    
    return model_60

In [105]:
tuner60 = RandomSearch(
    build_model_60,
    objective="val_loss",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory=os.path.normpath('C:/Runs'),
    project_name="10b-60",
)

In [106]:
tuner60.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 100, 'step': 1, 'sampling': 'linear'}
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
kernel_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [107]:
tuner60.search((Wt60,Xt60), yt60, epochs=200,
            validation_split = val_percent,
            callbacks=[early_stop],
            verbose=2)

Trial 10 Complete [01h 15m 17s]
val_loss: 1130.6312255859375

Best val_loss So Far: 1130.6312255859375
Total elapsed time: 14h 20m 56s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [108]:
tuner60.results_summary(1)

Results summary
Results in C:\Runs\10b-60
Showing 1 best trials
Objective(name="val_loss", direction="min")

Trial 09 summary
Hyperparameters:
units: 96
filters: 320
kernel_size: 3
learning_rate: 0.001
Score: 1130.6312255859375


### Best model

In [109]:
best_hp60 = tuner60.get_best_hyperparameters()[0]
model60 = tuner60.hypermodel.build(best_hp60)

In [110]:
scaler_filename = "Best_HP/10b-60"
joblib.dump(best_hp60, scaler_filename) 

['Best_HP/10b-60']

In [111]:
scaler_filename = "Best_HP/10b-60"
best_hp60 = joblib.load(scaler_filename) 

model60 = tuner60.hypermodel.build(best_hp60)

In [112]:
model60.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
DeepInput (InputLayer)          [(None, 3, 5, 15, 7) 0                                            
__________________________________________________________________________________________________
WideInput (InputLayer)          [(None, 3, 3, 7)]    0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 3, 3, 13, 320 20480       DeepInput[0][0]                  
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 3, 21)        0           WideInput[0][0]                  
____________________________________________________________________________________________

In [113]:
model60.fit((Wt60,Xt60), yt60, validation_split = val_percent,
            epochs = 100, 
            verbose = 1
            )

Epoch 1/100
777/777 [==============================] - 91s 103ms/step - loss: 18494.5723 - mean_absolute_error: 98.2682 - root_mean_squared_error: 135.9947 - mean_absolute_percentage_error: 61388516.0000 - val_loss: 3116.4819 - val_mean_absolute_error: 39.5535 - val_root_mean_squared_error: 55.8255 - val_mean_absolute_percentage_error: 5376163.5000
Epoch 2/100
777/777 [==============================] - 80s 103ms/step - loss: 1754.3810 - mean_absolute_error: 29.8845 - root_mean_squared_error: 41.8853 - mean_absolute_percentage_error: 67687272.0000 - val_loss: 1429.5996 - val_mean_absolute_error: 26.7572 - val_root_mean_squared_error: 37.8100 - val_mean_absolute_percentage_error: 5001290.0000
Epoch 3/100
777/777 [==============================] - 76s 97ms/step - loss: 1034.3583 - mean_absolute_error: 23.5606 - root_mean_squared_error: 32.1614 - mean_absolute_percentage_error: 64667424.0000 - val_loss: 1237.2972 - val_mean_absolute_error: 24.5127 - val_root_mean_squared_error: 35.1752 - v

777/777 [==============================] - 77s 99ms/step - loss: 748.1889 - mean_absolute_error: 19.9068 - root_mean_squared_error: 27.3530 - mean_absolute_percentage_error: 58211752.0000 - val_loss: 1130.2010 - val_mean_absolute_error: 22.7579 - val_root_mean_squared_error: 33.6185 - val_mean_absolute_percentage_error: 3812479.5000
Epoch 48/100
777/777 [==============================] - 78s 100ms/step - loss: 746.3524 - mean_absolute_error: 19.8984 - root_mean_squared_error: 27.3195 - mean_absolute_percentage_error: 58092552.0000 - val_loss: 1119.2191 - val_mean_absolute_error: 22.6916 - val_root_mean_squared_error: 33.4547 - val_mean_absolute_percentage_error: 3866293.2500370 - mean_absolute_error: 19.9438 - root_mean_squared_error: 27.4051 - mean_absolute_percentage_error: 62153100.00 - ETA: 18s - loss: 750.8375 - mean_absolute_e - ETA: 13s - loss: 747.7918 - mean_absolute_error: 19.9145 - root_m - ETA: 10s - loss: 748.70
Epoch 49/100
777/777 [==============================] - 80s 1

Epoch 70/100
777/777 [==============================] - 80s 103ms/step - loss: 732.4382 - mean_absolute_error: 19.7078 - root_mean_squared_error: 27.0636 - mean_absolute_percentage_error: 57912340.0000 - val_loss: 1124.8237 - val_mean_absolute_error: 22.6898 - val_root_mean_squared_error: 33.5384 - val_mean_absolute_percentage_error: 3926283.5000
Epoch 71/100
777/777 [==============================] - 77s 99ms/step - loss: 731.3774 - mean_absolute_error: 19.7008 - root_mean_squared_error: 27.0440 - mean_absolute_percentage_error: 57883492.0000 - val_loss: 1121.9508 - val_mean_absolute_error: 22.6777 - val_root_mean_squared_error: 33.4955 - val_mean_absolute_percentage_error: 3988576.7500
Epoch 72/100
777/777 [==============================] - 80s 103ms/step - loss: 730.8011 - mean_absolute_error: 19.6873 - root_mean_squared_error: 27.0333 - mean_absolute_percentage_error: 58087416.0000 - val_loss: 1119.6205 - val_mean_absolute_error: 22.6528 - val_root_mean_squared_error: 33.4607 - val

In [114]:
metrics60 = pd.DataFrame(model60.history.history)

In [115]:
# fig = plt.figure(figsize=(6,4))

# # Add set of axes to figure
# axes = fig.add_axes([0, 0, 1, 1]) 
# axes2 = fig.add_axes([0.3, 0.35, 0.5, 0.5]) # Smaller figure
# axes.plot(metrics60[['loss','val_loss']], label=['train_loss','val_loss'], lw=4)
# axes.set_ylim(0,4000)

# axes2.plot(metrics60[['loss','val_loss']], lw=4)
# axes2.set_xlim(0,20)
# axes2.set_ylim(500,1500)
# # plt.xticks(visible=False)
# plt.yticks(visible=False)

# axes.set_xlabel('Epoch')
# axes.set_ylabel('Loss')
# axes.grid()
# axes2.grid()

# mark_inset(axes, axes2, loc1=2, loc2=4, fc="0.9", ec="0.1", ls='--')
# axes.yaxis.set_major_formatter(formatter)

In [116]:
model60.save("my_dir/10b-60")

INFO:tensorflow:Assets written to: my_dir/10b-60\assets


INFO:tensorflow:Assets written to: my_dir/10b-60\assets


In [117]:
model60 = keras.models.load_model("my_dir/10b-60")

In [118]:
model60.predict((Wv60,Xv60))

array([[ 53.513737,  54.342228,  60.075054, ...,  59.130672,  37.20425 ,
         36.68805 ],
       [ 52.759792,  53.733692,  59.25777 , ...,  59.25864 ,  38.26328 ,
         37.85563 ],
       [ 51.46375 ,  52.392414,  57.793346, ...,  57.96033 ,  37.685223,
         37.33219 ],
       ...,
       [138.57323 , 142.54984 , 154.2424  , ..., 148.88919 , 101.34699 ,
         90.62144 ],
       [139.08858 , 143.23508 , 154.95642 , ..., 150.871   , 102.44961 ,
         92.283066],
       [124.22687 , 128.20653 , 138.31006 , ..., 132.52254 ,  88.16641 ,
         79.858826]], dtype=float32)

## Save Loss

In [ ]:
# metrics5.to_csv('Loss/8a-5.csv') 
# metrics15.to_csv('Loss/8a-15.csv') 
# metrics30.to_csv('Loss/8a-30.csv') 
# metrics60.to_csv('Loss/8a-60.csv') 